In [1]:
!pip install datasets
!pip install openpyxl
!pip install -q -U google-genai
# !pip install transformers
# !pip install accelerate
# !pip install peft
# !pip install bitsandbytes

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 111.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 136.0 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.5.1
    Uninstalling fsspec-2025.5.1:━━━━━━━━━━━━━━━━━━━━━━━━━━━━━  4/15 [fsspec]
      Successfully uninstalled fsspec-2025.5.1━━━━━━━━━━━━━━━━  4/15 [fsspec]
  Attempting uninstall: dill━━━━━━━━━━━━━━━━━━━━━━━━━━━━━  4/15 [fsspec]
    Found existing installation: dill 0.4.0━━━━━━━━━━━━━━━━━━━━━━━  6/15 [dill]
    Uninstalling dill-0.4.0:m╺━━━━━━━━━━━━━━━━━━━━━━━  6/15 [dill]
      Successfully uninstalled dill-0.4.00m━━━━━━━━━━━━━━━━━━━━━━━  6/15 [dill]
  Attempting uninstall: multiprocess0m━━━━━━━━━━━━━━━━━━━━━━━  6/15 [dill]
    Found existing installation: multiprocess 0.70.18━━━━━━━━━  6/15 [dill]
    Uninstalling multiprocess-0.70.18:╸━━━━━━━━━━━━━ 10/15 [multiprocess]
      Successfully uninstalled multiprocess-0.70.18━━━━━━━━━━━ 10/15 [multiprocess]

# Imports

In [45]:
%load_ext autoreload
%autoreload 2

import pandas as pd
import sys

# Call models
from src.call_models import bedrock_connect, call_claude_bedrock
from src.call_models import google_connect, call_gemini, all_string_gemini_config, all_int_gemini_config
from src.translate_func import claude_translation, gemini_translation, gemini_multi_translation, gemini_claude_best_translation

# Datasets
from src.benchmarks_code import arc_ai
from prompts import arc_prompts

# Access keys
from my_access_keys import google_access_key, aws_access_key, aws_secret_key

# .csv utils
from src.save_utils import add_dataset_to_csv

# Remove annoying warning
from IPython.core.display_functions import display

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [5]:
# Get the model's access keys
bedrock_client = bedrock_connect(aws_access_key, aws_secret_key)
google_client = google_connect(google_access_key)

# Check Models Calls

In [52]:
print('claude:')
print(call_claude_bedrock(
    bedrock_client,
    'Which one is more accurate in Hebrew? repeat the right sentence, without adding anything\nהיווצרות יבשת העל פנגיאה OR היווצרות היבשת העל פנגיאה',
    system_prompt='ALWAYS start and end your answers with "*"',
))

claude:
*היווצרות יבשת העל פנגיאה*


# ARC_AI2

## Get Datset

In [6]:
arc_dataset = arc_ai.get_arc_ai2_datasets()
arc_dataset['arc_challenge_train'] = arc_dataset['arc_challenge_train'].skip(5).take(50)
file_name = 'compare_csv/arc_ai2_chall_train_top_50.csv'

print(file_name)
arc_dataset

README.md: 0.00B [00:00, ?B/s]

ARC-Challenge/train-00000-of-00001.parqu(…):   0%|          | 0.00/190k [00:00<?, ?B/s]

ARC-Challenge/test-00000-of-00001.parque(…):   0%|          | 0.00/204k [00:00<?, ?B/s]

ARC-Challenge/validation-00000-of-00001.(…):   0%|          | 0.00/55.7k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/1119 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1172 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/299 [00:00<?, ? examples/s]

compare_csv/arc_ai2_chall_train_top_50.csv


{'arc_challenge_train': Dataset({
     features: ['id', 'question', 'choices', 'answerKey'],
     num_rows: 50
 })}

In [7]:
df = pd.read_csv(file_name)
df.head(3)

,original,claude_v1,claude_v2,claude_v3,claude_v4,claude_v5,claude_v6,gemini_v1,gemini_pro_v1,gemini_v2,gemini_pro_v2,gemini_v2_think,gemini_v3,gemini_v3_think
0,<question>Which land form is the result of the...,<question>איזו צורת קרקע היא תוצאה של הכוח הבו...,<question>איזו צורת קרקע היא תוצאה של הכוח הבו...,<question>איזו צורת קרקע היא תוצאה של הכוח הבו...,<question>איזו צורת קרקע היא תוצאה של הכוח הבו...,<question>איזו צורת קרקע היא תוצאה של הכוח הבו...,<question>איזו צורת קרקע היא תוצאה של הכוח הבו...,<question>איזו צורת נוף היא תוצאה של הכוח הבונ...,<question>איזו תצורת נוף היא תוצאה של הכוח הבו...,<question>איזו צורת נוף היא תוצאה של הכוח הבונ...,<question>איזו צורת נוף היא תוצאה של כוח בונה ...,<question>איזו צורת נוף היא תוצאה של הכוח הבונ...,<question>איזו צורת נוף היא תוצאה של כוח בונה ...,<question>איזו צורת נוף היא תוצאה של הכוח הבונ...
1,<question>Which statement best compares single...,<question>איזה היגד משווה בצורה הטובה ביותר בי...,<question>איזה היגד משווה בצורה הטובה ביותר בי...,<question>איזה היגד משווה בצורה הטובה ביותר בי...,<question>איזה היגד משווה בצורה הטובה ביותר בי...,<question>איזה היגד משווה בצורה הטובה ביותר בי...,<question>איזה היגד משווה בצורה הטובה ביותר בי...,<question>איזה היגד משווה בצורה הטובה ביותר או...,<question>איזה היגד משווה בצורה הטובה ביותר בי...,<question>איזו טענה משווה בצורה הטובה ביותר בי...,<question>איזה היגד משווה בצורה הטובה ביותר בי...,<question>איזה היגד משווה בצורה הטובה ביותר בי...,<question>איזה היגד משווה בצורה הטובה ביותר בי...,<question>איזה היגד משווה בצורה הטובה ביותר בי...
2,"<question>As part of an experiment, an astrona...","<question>כחלק מניסוי, אסטרונאוט לוקח משקל ליר...","<question>כחלק מניסוי, אסטרונאוט לוקח משקל ליר...","<question>כחלק מניסוי, אסטרונאוט לוקח משקל ליר...","<question>כחלק מניסוי, אסטרונאוט לוקח משקל ליר...","<question>כחלק מניסוי, אסטרונאוט לוקח משקל ליר...","<question>כחלק מניסוי, אסטרונאוט לוקח משקל ליר...","<question>כחלק מניסוי, אסטרונאוט לוקח משקל ליר...","<question>כחלק מניסוי, אסטרונאוט לוקח משקל ליר...","<question>כחלק מניסוי, אסטרונאוט לוקח מאזניים ...","<question>כחלק מניסוי, אסטרונאוט לוקח משקל ליר...","<question>כחלק מניסוי, אסטרונאוט לוקח מאזניים ...","<question>כחלק מניסוי, אסטרונאוט לוקח מאזניים ...","<question>כחלק מניסוי, אסטרונאוט לוקח מאזניים ..."


## Run Translation

### Claude

In [91]:
exp_name = 'claude_v6'

hebrew_datasets = claude_translation(
    bedrock_client,
    arc_dataset,
    arc_prompts.ARC_INSTRUCT_V6_CLAUDE,
    arc_prompts.ARC_FEW_SHOTS,
    arc_prompts.ARC_FORMAT,
    arc_ai.arc_sample_to_dict,
    arc_ai.arc_dict_to_sample,
)

Translating arc_challenge_train...


  0%|          | 0/50 [00:00<?, ?it/s]

In [5]:
df = add_dataset_to_csv(file_name, exp_name, hebrew_datasets['arc_challenge_train'], arc_ai.arc_sample_to_dict)
df.head(2)

,original,claude_v1,claude_v2,claude_v3,claude_v4,claude_v5,claude_v6
0,<question>Which land form is the result of the...,<question>איזו צורת קרקע היא תוצאה של הכוח הבו...,<question>איזו צורת קרקע היא תוצאה של הכוח הבו...,<question>איזו צורת קרקע היא תוצאה של הכוח הבו...,<question>איזו צורת קרקע היא תוצאה של הכוח הבו...,<question>איזו צורת קרקע היא תוצאה של הכוח הבו...,<question>איזו צורת קרקע היא תוצאה של הכוח הבו...
1,<question>Which statement best compares single...,<question>איזה היגד משווה בצורה הטובה ביותר בי...,<question>איזה היגד משווה בצורה הטובה ביותר בי...,<question>איזה היגד משווה בצורה הטובה ביותר בי...,<question>איזה היגד משווה בצורה הטובה ביותר בי...,<question>איזה היגד משווה בצורה הטובה ביותר בי...,<question>איזה היגד משווה בצורה הטובה ביותר בי...


### Gemini

In [21]:
exp_name = 'gemini_v1_think'

hebrew_datasets = gemini_translation(
    google_client,
    arc_dataset,
    arc_prompts.ARC_INSTRUCT_V1_GEMINI,
    arc_prompts.ARC_FEW_SHOTS,
    # arc_prompts.ARC_SYSTEM_V2_GEMINI,
    # arc_prompts.ARC_INSTRUCT_V3_GEMINI + '\n\n' + arc_prompts.ARC_FEW_SHOTS,
    arc_ai.arc_sample_to_dict,
    arc_ai.arc_dict_to_sample,
    if_pro=False,
    think_bud=10_000
)

Translating arc_challenge_train...


  0%|          | 0/50 [00:00<?, ?it/s]

Sleeping.... Done!                                

In [ ]:
hebrew_datasets

In [23]:
df = add_dataset_to_csv(file_name, exp_name, hebrew_datasets['arc_challenge_train'], arc_ai.arc_sample_to_dict)
df.head(2)

,original,claude_v1,claude_v2,claude_v3,claude_v4,claude_v5,claude_v6,gemini_v1,gemini_pro_v1,gemini_v2,gemini_pro_v2,gemini_v2_think,gemini_v3,gemini_v3_think,gemini_pro_v1_think,gemini_v1_think
0,<question>Which land form is the result of the...,<question>איזו צורת קרקע היא תוצאה של הכוח הבו...,<question>איזו צורת קרקע היא תוצאה של הכוח הבו...,<question>איזו צורת קרקע היא תוצאה של הכוח הבו...,<question>איזו צורת קרקע היא תוצאה של הכוח הבו...,<question>איזו צורת קרקע היא תוצאה של הכוח הבו...,<question>איזו צורת קרקע היא תוצאה של הכוח הבו...,<question>איזו צורת נוף היא תוצאה של הכוח הבונ...,<question>איזו תצורת נוף היא תוצאה של הכוח הבו...,<question>איזו צורת נוף היא תוצאה של הכוח הבונ...,<question>איזו צורת נוף היא תוצאה של כוח בונה ...,<question>איזו צורת נוף היא תוצאה של הכוח הבונ...,<question>איזו צורת נוף היא תוצאה של כוח בונה ...,<question>איזו צורת נוף היא תוצאה של הכוח הבונ...,<question>איזו תצורת נוף היא תוצאה של הכוח הבו...,<question>איזו צורת נוף היא תוצאה של הכוח הבונ...
1,<question>Which statement best compares single...,<question>איזה היגד משווה בצורה הטובה ביותר בי...,<question>איזה היגד משווה בצורה הטובה ביותר בי...,<question>איזה היגד משווה בצורה הטובה ביותר בי...,<question>איזה היגד משווה בצורה הטובה ביותר בי...,<question>איזה היגד משווה בצורה הטובה ביותר בי...,<question>איזה היגד משווה בצורה הטובה ביותר בי...,<question>איזה היגד משווה בצורה הטובה ביותר או...,<question>איזה היגד משווה בצורה הטובה ביותר בי...,<question>איזו טענה משווה בצורה הטובה ביותר בי...,<question>איזה היגד משווה בצורה הטובה ביותר בי...,<question>איזה היגד משווה בצורה הטובה ביותר בי...,<question>איזה היגד משווה בצורה הטובה ביותר בי...,<question>איזה היגד משווה בצורה הטובה ביותר בי...,<question>איזה היגד משווה בצורה הטובה ביותר בי...,<question>איזה היגד משווה בצורה הטובה ביותר או...


### Multi-options Translation - Claude

### Multi-options Translation - Gemini

In [ ]:
print(arc_prompts.ARC_INSTRUCT_MULTI_V1)

In [ ]:
hebrew_datasets = gemini_multi_translation(
    google_client,
    arc_dataset,
    arc_prompts.ARC_INSTRUCT_MULTI_V1,
    arc_prompts.ARC_FEW_SHOTS,
    arc_ai.arc_sample_to_dict,
    arc_ai.arc_dict_to_sample,
)

___

### Claude vs Gemini (using Gemini as judge)

In [39]:
df.columns

Index(['original', 'claude_v1', 'claude_v2', 'claude_v3', 'claude_v4',
       'claude_v5', 'claude_v6', 'gemini_v1', 'gemini_pro_v1', 'gemini_v2',
       'gemini_pro_v2', 'gemini_v2_think', 'gemini_v3', 'gemini_v3_think',
       'gemini_pro_v1_think', 'gemini_v1_think'],
      dtype='object')

In [40]:
best_claude = 'claude_v6'
# best_gemini = 'gemini_v1_think'
best_gemini = 'gemini_pro_v1_think'

In [ ]:
hebrew_datasets = gemini_claude_best_translation(
    google_client,
    bedrock_client,
    arc_dataset,
    arc_prompts.ARC_INSTRUCT_V,
    arc_prompts.ARC_FEW_SHOTS,
    arc_prompts.ARC_FORMAT,
    arc_ai.arc_sample_to_dict,
    arc_ai.arc_dict_to_sample,
)

# GSM8K

In [ ]:
"""<instruction>
Your task is to translate the given English question and answer into Hebrew, following these guidelines:

<guidelines>
1. Only translate the provided question and answer. Do not add any additional text or instructions.
2. Preserve the original meaning and intent of the question and answer as accurately as possible in the Hebrew translation.
3. Maintain the same structure and formatting as the original English version, including bullet points, numbering, or other formatting elements.
</guidelines>

<example_format>
Provide the Hebrew translation immediately after these instructions in the following format:

<question>
Translated question
</question>
<answer>
Translated answer
</answer>
</example_format>
</instruction>

<fewshot_examples>
<example>
English:
<question>
In April, Tank gathered 10 more Easter eggs than Emma in their first round of egg hunt. However, Emma gathered twice as many eggs as Tank in their second round of egg hunt, while Tank's total number of eggs in the second round was 20 less than the number she had gathered in the first round. If the total number of eggs in the pile they were collecting with 6 other people was 400 eggs, and Emma gathered 60 eggs in the second egg hunt round, find the number of eggs that the 6 other egg hunters collected?
</question>
<answer>
Tank's total number of eggs in the second round was 60/2=<<60/2=30>>30 since Emma gathered twice as many eggs as Tank in their second round of egg hunt.
The total number of eggs that Emma and Tank gathered in the second round was 60+30=<<60+30=90>>90
Tank's total number of eggs in the second round was 20 less than the number she had gathered in the first round, meaning she had gathered 30+20=<<30+20=50>>50 eggs in the first round of egg hunt.
Tank gathered 10 more Easter eggs than Emma in their first round of egg hunt, meaning Emma collected 50-10=40 eggs
The total number of eggs Emma and Tank collected in the first round was 40+50=<<40+50=90>>90
In the two rounds, Emma and Tank gathered 90+90=<<90+90=180>>180 eggs
If the total number of eggs in the pile they were collecting with 6 other people was 400 eggs, the six other people gathered 400-180=<<400-180=220>>220 eggs
#### 220
</answer>

Hebrew:
<question>
באפריל, טאנק אספה 10 ביצי פסחא יותר מאמה בסבב הראשון של ציד ביצים. עם זאת, אמה אספה פי שניים ביצים מטאנק בסבב השני של ציד ביצים, בעוד שהמספר הכולל של הביצים של טאנק בסבב השני היה 20 פחות מהמספר שהיא אספה בסבב הראשון. אם המספר הכולל של הביצים בערימה שהן אספו יחד עם 6 אנשים נוספים היה 400 ביצים, ואמה אספה 60 ביצים בסבב השני של ציד ביצים, מצאו את מספר הביצים שאספו 6 ציידי הביצים האחרים?
</question>
<answer>
מספר ביצי הפסחא הכולל של טאנק בסיבוב השני היה 60/2=<<60/2=30>>30 מכיוון שאמה אספה פי שניים ביצי פסחא מטנק בסיבוב השני שלהן בציד הביצים.
מספר ביצי הפסחא הכולל שאמה וטאנק אספו בסיבוב השני היה 60+30=<<60+30=90>>90
מספר ביצי הפסחא הכולל של טאנק בסיבוב השני היה 20 פחות ממספר הביצים שהיא אספה בסיבוב הראשון, כלומר היא אספה 30+20=<<30+20=50>>50 ביצי פסחא בסיבוב הראשון של ציד הביצים.
טאנק אספה 10 ביצי פסחא יותר מאמה בסיבוב הראשון שלהן בציד הביצים, כלומר אמה אספה 50-10=40 ביצי פסחא
מספר ביצי הפסחא הכולל שאמה וטאנק אספו בסיבוב הראשון היה 40+50=<<40+50=90>>90
בשני הסיבובים, אמה וטאנק אספו 90+90=<<90+90=180>>180 ביצי פסחא
אם המספר הכולל של ביצי הפסחא בערימה שהן אספו יחד עם 6 אנשים אחרים היה 400 ביצים, ששת האנשים האחרים אספו 400-180=<<400-180=220>>220 ביצי פסחא
#### 220
</answer>
</example>
</fewshot_examples>

"""